In [105]:
# Package imports
import os
import glob
# import boto3
# import sagemaker
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### Import Data Subset

In [106]:
file_path = '../data_complete'

In [107]:
# Read Rating and Book Meta Data
book_rating = []
for file in glob.glob(file_path + "./book*.csv"):
    df = pd.read_csv(file)
    book_rating.append(df)

        
user_rating = []
for file in glob.glob(file_path + "./user_rating*.csv"):
    df = pd.read_csv(file)
    user_rating.append(df)

user_rating_df = pd.concat(user_rating)
book_rating_df = pd.concat(book_rating)

## Data Overview

For the purposes of this example, only a smaller subset of the entire dataset - for both user ratings and book metadata - is used. 

### Explore User Ratings
User ratings contains 3 columns:
- a User ID
- Name of the book
- Rating, one of {'it was amazing', 'really liked it', 'liked it', 'did not like it','it was ok', "This user doesn't have any rating"}

In [108]:
user_rating_df['Rating'].unique()

array(['it was amazing', 'really liked it', 'liked it', 'did not like it',
       'it was ok', "This user doesn't have any rating"], dtype=object)

### Mapping ratings to numeric values (ordinal)

In [109]:
oe = preprocessing.OrdinalEncoder(categories =[["This user doesn't have any rating",'did not like it','it was ok','liked it','really liked it','it was amazing']])
user_rating_df['Rating_numeric'] = oe.fit_transform(user_rating_df.loc[:,['Rating']])

In [110]:
user_rating_df.head()

,Unnamed: 0.1,Unnamed: 0,ID,Name,Rating,Rating_numeric
0,0.0,0.0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5.0
1,1.0,1.0,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5.0
2,2.0,2.0,1,Siddhartha,it was amazing,5.0
3,3.0,3.0,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4.0
4,4.0,4.0,1,"Ready Player One (Ready Player One, #1)",really liked it,4.0


In [111]:
user_rating_df.to_csv(file_path + '/user_ratings.csv')

### Retaining Users with a Minimum number of ratings
In order to generate meaningful results, we need to ensure that the retained users have rated at least a minimum number of books. For this round of analysis, this threshold is set to 5 books.

In [112]:
#filtering users with rating > 0 (actual rating provided)
pos_user_rating = user_rating_df.loc[user_rating_df['Rating_numeric']>0]
pos_user_rating.rename(columns={'ID':'user_id'}, inplace=True)


C:\Users\shett\AppData\Local\Temp\ipykernel_14868\1129833459.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_user_rating.rename(columns={'ID':'user_id'}, inplace=True)


In [113]:
# Getting a count of ratings by user
user_rating_summary = pos_user_rating[['user_id', 'Rating_numeric']].groupby(['user_id']).agg(['count']).reset_index()
user_rating_summary.columns = ['_'.join(col) for col in user_rating_summary.columns.values]

In [114]:
kk = (len(user_rating_summary[user_rating_summary['Rating_numeric_count']>=5])/len(user_rating_summary))*100
print("Percentage of people with 5 or more reviews: "+ str(kk))

Percentage of people with 5 or more reviews: 82.85989407799711


In [115]:
# Retaining users with 5 ratings or more
frequent_user_list = user_rating_summary.loc[user_rating_summary['Rating_numeric_count']>=5]['user_id_'].to_list()
filtered_pos_user_rating = pos_user_rating.loc[pos_user_rating['user_id'].isin(frequent_user_list)]

In [116]:
# Filtering any duplicates
filtered_pos_user_rating = filtered_pos_user_rating.drop_duplicates(subset=['Name','user_id'], keep="first")

In [117]:
print('FInal count of unique users is '+str(filtered_pos_user_rating['user_id'].nunique()))

FInal count of unique users is 3442


### Clean the book rating data

The book ratings dataset includes the average ratings data for books along with the distribution of ratings (1s, 2s, 3s etc.). A lot of the books also seem to have duplicate entries - with different ISBNs, in some cases - different publishers etc.  
One such example is Wuthering Heights below.

In [118]:
book_rating_df.loc[book_rating_df['Name']=='Wuthering Heights']

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
3814,6183,Wuthering Heights,1:56790,248.0,4:390456,total:1270598,1,3,Running Press Book Publishers,94,...,NaN,Emily Brontë,3.85,2:102674,5:454302,0762405597,3:266376,NaN,NaN,NaN
50983,87798,Wuthering Heights,1:56937,335.0,4:391895,total:1275195,1,1,Penguin Books,114,...,eng,Emily Brontë,3.85,2:102963,5:456212,0140434186,3:267188,NaN,NaN,NaN
35231,1088194,Wuthering Heights,1:57809,158.0,4:400255,total:1301773,1,1,Usborne Books,4,...,NaN,Emily Brontë,3.86,2:104729,5:466772,1580866042,3:272208,"- Classic adaptations with adventure, exciteme...",4.0,NaN
22642,138857,Wuthering Heights,1:56957,688.0,4:392068,total:1275733,1,12,Kaplan Publishing,2,...,NaN,Emily Brontë,3.85,2:102999,5:456405,0743261992,3:267304,NaN,NaN,NaN
29366,1171102,Wuthering Heights,1:1,96.0,4:5,total:23,1,9,Penguin Books Ltd,0,...,NaN,John Escott,3.74,2:4,5:9,0140814221,3:4,[<i>Simply Stories Level 5</i>]<br /><br />An ...,0.0,NaN
4408,1210658,Wuthering Heights,1:57898,4.0,4:400807,total:1303702,1,3,HarperCollins UK,1,...,NaN,Emily Brontë,3.86,2:104874,5:467552,0001046403,3:272571,"First published in 1847, Emily Bronte's classi...",1.0,NaN
9855,1324394,Wuthering Heights,1:0,83.0,4:19,total:54,1,8,Barron's Educational Series,4,...,NaN,Jane Easton,4.06,2:2,5:20,0764108298,3:13,Six more volumes in this recently developed se...,4.0,NaN
12786,1436006,Wuthering Heights,1:58012,346.0,4:401821,total:1306966,1,1,Könemann,1,...,NaN,Emily Brontë,3.86,2:105092,5:468874,3895082082,3:273167,"Set on the stormy moors of northern England, t...",1.0,NaN
24448,1572940,Wuthering Heights,1:58078,392.0,4:402363,total:1308677,1,3,Dutton Books,0,...,eng,Emily Brontë,3.86,2:105195,5:469549,0460002430,3:273492,Published in 1847 and initially considered exc...,0.0,NaN
9371,2021462,Wuthering Heights,1:58418,NaN,4:405885,total:1319698,10,5,Tark Classic Fiction,20,...,NaN,Emily Brontë,3.86,2:105966,5:473864,160450028X,3:275565,Emily Bronte's only novel appeared to mixed re...,NaN,252.0


In [119]:
# reatiing only English books
eng_lang_list = ['eng', 'en-US', 'en-GB', 'en-CA']
book_rating_df = book_rating_df.loc[book_rating_df['Language'].isin(eng_lang_list)]

### Handle duplicate entries
There are a few different ways of handling duplicates.  
A simplified way of handling duplicate book entries can be to average out the ratings (using a weighted average and a count of ratings) to create a unique entry.

However, for the first cut (and to keep things really simple), we will retain just the entry with the most number of reviews.

In [120]:
book_rating_df = book_rating_df.sort_values(by=['Name','CountsOfReview'], ascending=[True, False])
book_rating_df_unique = book_rating_df.drop_duplicates(subset='Name', keep="first")

In [121]:
book_rating_df_unique.loc[book_rating_df_unique['Name']=='Wuthering Heights']

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
50983,87798,Wuthering Heights,1:56937,335.0,4:391895,total:1275195,1,1,Penguin Books,114,...,eng,Emily Brontë,3.85,2:102963,5:456212,0140434186,3:267188,NaN,NaN,NaN


In [122]:
# Rename id column
book_rating_df_unique.rename(columns={'Id':'book_id'}, inplace=True)
print('Unique count of books is ' + str(book_rating_df_unique['book_id'].nunique()))

Unique count of books is 184420


C:\Users\shett\AppData\Local\Temp\ipykernel_14868\243215339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_rating_df_unique.rename(columns={'Id':'book_id'}, inplace=True)


### Merging with user dataset

In [123]:
len(filtered_pos_user_rating)

356269

In [124]:
filtered_pos_user_rating.head()

,Unnamed: 0.1,Unnamed: 0,user_id,Name,Rating,Rating_numeric
0,0.0,0.0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5.0
1,1.0,1.0,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5.0
2,2.0,2.0,1,Siddhartha,it was amazing,5.0
3,3.0,3.0,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4.0
4,4.0,4.0,1,"Ready Player One (Ready Player One, #1)",really liked it,4.0


In [125]:
book_rating_df.head(2)

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
244421,2588957,"!Musica!: Salsa, Rumba, Merengue, & More: The ...",1:0,NaN,4:7,total:13,1,1,Diane Pub Co,0,...,eng,Sue Steward,3.69,2:0,5:1,NaN,3:5,"Salsa, the sexy dance music of the Spanish-spe...",NaN,176.0
4941,911352,""" Memories of Underdevelopment"" and ""Inconsola...",1:0,272.0,4:8,total:19,1,6,Rutgers University Press,2,...,eng,Edmundo Desnoes,3.95,2:2,5:6,0813515378,3:3,Memories of Underdevelopment was the first gre...,2.0,NaN


In [126]:
merged_user_ratings = pd.merge(filtered_pos_user_rating, book_rating_df_unique[['book_id','Name']], on = 'Name', how ='left')
merged_user_ratings.dropna(subset='book_id', inplace=True)
merged_user_ratings.to_csv(file_path + '/merged_user_ratings.csv')

In [127]:
user_rating_reshape = filtered_pos_user_rating.pivot(index = 'user_id', columns ='Name', values = 'Rating_numeric').fillna(0)
user_rating_reshape.head()

Name,!آنچه سینما هست,!از قر و قمبیل‌های قلمی بی‌قال و قیل,""" Talking Heads ""","""A Problem from Hell"": America and the Age of Genocide","""A"" Is for Africa","""A"" is for Apple (Sophie Green Mystery, #3)","""B"" Is for Betsy","""Beat"" Takeshi Kitano","""C"" Is For Corpse (Kinsey Millhone #3)","""Cinema Paradiso""",...,"달빛 조각사 42 (The Legendary Moonlight Sculptor, #42)","달빛 조각사 43 (The Legendary Moonlight Sculptor, #43)","달빛 조각사 5 (The Legendary Moonlight Sculptor, #5)","달빛 조각사 6 (The Legendary Moonlight Sculptor, #6)","달빛 조각사 7 (The Legendary Moonlight Sculptor, #7)","달빛 조각사 8 (The Legendary Moonlight Sculptor, #8)","달빛 조각사 9 (The Legendary Moonlight Sculptor, #9)",모리와 함께한 화요일,흰,�come Closer�: Critical Perspectives on Theatre of the Oppressed
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Save relevant datasets

In [129]:
#convert to numpy matrix
user_rating_matrix = user_rating_reshape.to_numpy()
np.savetxt(file_path +'/user_rating_matrix.txt', user_rating_matrix, fmt='%d')

In [130]:
book_rating_df_unique.to_csv(file_path + '/book_detils.csv')